## 1. Clima - Open Meteo

### 1.1. Configuración Inicial y Carga de Datos

Cargamos las librerías necesarias y configuramos las rutas del proyecto usando el archivo central `src/config.py`.

In [1]:
import pandas as pd
from pathlib import Path
import sys
import os
 
# ----------------------------------------------------------------------
# A. ARREGLO DE PATH PARA IMPORTAR MÓDULOS DEL PROYECTO (SRC)
# ----------------------------------------------------------------------

# Intentamos subir hasta que encontremos la carpeta 'src'
# Esto hace que la importación funcione sin importar dónde esté abierta la libreta.
current_dir = Path().resolve()
project_root = current_dir

# Buscamos la raíz del proyecto (donde está la carpeta 'src')
for _ in range(5):
    if (project_root / 'src').is_dir():
        break
    project_root = project_root.parent
else:
    # Esta excepción ya no debería saltar después de nuestro arreglo de PATH
    raise FileNotFoundError("No se pudo encontrar la carpeta 'src' en los directorios superiores. Verifica la estructura del proyecto.")

# Añadir la raíz del proyecto al path de Python
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
# ----------------------------------------------------------------------

# B. IMPORTAR HERRAMIENTAS Y CONFIGURACIÓN

# Ahora importamos PROCESSED_DIR en lugar de INTERIM_DIR
from src.config import RAW_DIR, PROCESSED_DIR, get_logger, init_paths 

# Inicializar logger
logger = get_logger(__name__)

# Asegurar que las carpetas existan
init_paths()

# Definir la ruta del archivo a cargar y guardar
INPUT_FILE = RAW_DIR / "clima_hermosillo.csv"

# ¡CAMBIO AQUÍ! Guardamos en la carpeta PROCESSED
OUTPUT_FILE = PROCESSED_DIR / "clima_final_processed.csv"

print("Rutas, logger y PATH inicializados correctamente.")

Rutas, logger y PATH inicializados correctamente.


## 1.2. Carga del Dataset e Inspección Inicial

El primer paso en la fase de limpieza es cargar el archivo de datos sin procesar (`clima_hermosillo.csv`) desde la carpeta `data/raw`.

Una vez cargado, realizaremos una inspección inicial para verificar:

* **Forma y Volumen:** Número total de filas y columnas del *dataset*.
* **Estructura:** Las primeras filas para confirmar que los datos se cargaron correctamente.
* **Tipos de Datos (`dtypes`):** Confirmar que Pandas está leyendo correctamente las columnas. Observamos que la columna **`date`** es un `object` (string), lo cual deberá ser corregido para trabajar con series de tiempo.

In [2]:
# Cargamos el archivo CSV y verificamos la integridad de la carga.

try:
    df_clima = pd.read_csv(INPUT_FILE)
    print(f"Dataset cargado exitosamente. Filas: {len(df_clima)}, Columnas: {len(df_clima.columns)}")

    print("\n--- Primeras 5 Filas ---")
    display(df_clima.head())
    
    print("\n--- Estructura y Tipos de Datos (dtypes) ---")
    df_clima.info()

except FileNotFoundError:
    logger.error(f"ERROR: El archivo no fue encontrado en la ruta: {INPUT_FILE}")
    logger.error("Asegúrate de haber ejecutado previamente el script 'download_clima.py'.")
    df_clima = pd.DataFrame()

Dataset cargado exitosamente. Filas: 26280, Columnas: 8

--- Primeras 5 Filas ---


,date,temperature_2m,precipitation,weather_code,is_day,relative_humidity_2m,cloud_cover,wind_speed_10m
0,2021-01-01 00:00:00+00:00,18.541000,0.0,0.0,1.0,16.600382,0.0,12.313894
1,2021-01-01 01:00:00+00:00,16.541000,0.0,0.0,0.0,18.832726,0.0,12.229406
2,2021-01-01 02:00:00+00:00,15.240999,0.0,0.0,0.0,22.180996,0.0,12.979984
3,2021-01-01 03:00:00+00:00,13.891000,0.0,0.0,0.0,24.950367,0.0,9.779817
4,2021-01-01 04:00:00+00:00,12.441000,0.0,0.0,0.0,29.475586,0.0,8.913181



--- Estructura y Tipos de Datos (dtypes) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  26280 non-null  object 
 1   temperature_2m        26280 non-null  float64
 2   precipitation         26280 non-null  float64
 3   weather_code          26280 non-null  float64
 4   is_day                26280 non-null  float64
 5   relative_humidity_2m  26280 non-null  float64
 6   cloud_cover           26280 non-null  float64
 7   wind_speed_10m        26280 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1.6+ MB


## 1.3. Conversión de Tipos de Datos y Revisión de Integridad

Esta sección se enfoca en preparar la columna temporal para el análisis y asegurar la integridad de los datos.

### Procesos clave

* **Conversión a `datetime`:** Transformamos la columna `'date'` y la establecemos como **índice** del DataFrame (esencial para series de tiempo).
* **Verificación de Nulos:** Contamos los valores nulos. Si existen, aplicamos la técnica de imputación **`ffill`** (*Forward Fill*), que es apropiada para datos climáticos, asumiendo que el último valor registrado es el más probable.

In [3]:
if not df_clima.empty:
    
    try:
        df_clima = pd.read_csv(INPUT_FILE)
    except Exception as e:
        logger.error(f"Error al recargar el CSV para la conversión: {e}")
        

    # 1. Convertir la columna 'date' a datetime
    df_clima['date'] = pd.to_datetime(df_clima['date'])

    # 2. Establecer 'date' como índice (esencial para series de tiempo)
    df_clima = df_clima.set_index('date')
    print("Columna 'date' convertida a tipo datetime y establecida como índice.")

    # 3. Revisión de Nulos
    nulos_por_columna = df_clima.isnull().sum()
    nulos_existentes = nulos_por_columna[nulos_por_columna > 0]
    
    if not nulos_existentes.empty:
        logger.warning("Se encontraron nulos. Aplicando imputación 'ffill'.")
        # Para datos climáticos, forward fill (ffill) es la estrategia común
        df_clima.fillna(method='ffill', inplace=True)
        print(f"Nulos restantes: {df_clima.isnull().sum().sum()}")
    else:
        print("No se detectaron valores nulos. La integridad es buena.")

else:
    logger.warning("DataFrame vacío. Omitiendo conversión y revisión de nulos.")

Columna 'date' convertida a tipo datetime y establecida como índice.
No se detectaron valores nulos. La integridad es buena.


## 1.4. Estandarización de Variables y Formato Final

Una vez que la integridad de los datos (`dtypes` y nulos) está asegurada, el último paso es hacer el *dataset* legible y utilizable, lo cual es vital antes de la fusión con otros *datasets*.

### Procesos clave

* **Renombrado:** Cambiamos los nombres técnicos de la API (e.g., `temperature_2m`) a nombres claros en español (e.g., `temperatura`).
* **Redondeo:** Estandarizamos todas las columnas de punto flotante a **un solo decimal**. Esto asegura consistencia y simplifica el análisis visual, ya que las mediciones de origen ya tienen suficiente precisión.

In [4]:
# Definición del mapeo de nombres para mayor claridad
columnas_renombrar = {
    'temperature_2m': 'temperatura',
    'precipitation': 'precipitacion',
    'weather_code': 'codigo_clima',
    'is_day': 'es_de_dia',
    'relative_humidity_2m': 'humedad',
    'cloud_cover': 'nubosidad',
    'wind_speed_10m': 'velocidad_viento'
}

# 1. Renombrar las columnas
df_clima.rename(columns=columnas_renombrar, inplace=True)
print("Columnas renombradas a español.")

# 2. Redondear todas las columnas numéricas a un decimal
# Excluimos columnas categóricas (como 'es_de_dia' que es binaria) para evitar errores.
columnas_a_redondear = ['temperatura', 'precipitacion', 'humedad', 'nubosidad', 'velocidad_viento']

for col in columnas_a_redondear:
    if col in df_clima.columns:
        # Usamos round(1) para un decimal
        df_clima[col] = df_clima[col].round(1)

print("Valores numéricos redondeados a un decimal para estandarización.")

# 3. Inspección final de la limpieza
print("\n--- Estructura final del DataFrame limpio ---")
display(df_clima.head())
print(df_clima.dtypes)

Columnas renombradas a español.
Valores numéricos redondeados a un decimal para estandarización.

--- Estructura final del DataFrame limpio ---


,temperatura,precipitacion,codigo_clima,es_de_dia,humedad,nubosidad,velocidad_viento
date,,,,,,,
2021-01-01 00:00:00+00:00,18.5,0.0,0.0,1.0,16.6,0.0,12.3
2021-01-01 01:00:00+00:00,16.5,0.0,0.0,0.0,18.8,0.0,12.2
2021-01-01 02:00:00+00:00,15.2,0.0,0.0,0.0,22.2,0.0,13.0
2021-01-01 03:00:00+00:00,13.9,0.0,0.0,0.0,25.0,0.0,9.8
2021-01-01 04:00:00+00:00,12.4,0.0,0.0,0.0,29.5,0.0,8.9


temperatura         float64
precipitacion       float64
codigo_clima        float64
es_de_dia           float64
humedad             float64
nubosidad           float64
velocidad_viento    float64
dtype: object


## 1.5. Guardar el Dataset Procesado (Finalización de Limpieza)

Una vez que el DataFrame ha sido transformado (conversión de tipos, gestión de nulos, renombrado y redondeo), el *dataset* de clima está listo.

Lo guardamos en la carpeta **`data/processed`** para indicar que está en su formato final y listo para el análisis exploratorio (EDA) o para la fusión con las otras fuentes (`INEGI` y `Bachómetro`).

In [5]:
if not df_clima.empty:
    # Se guarda el DataFrame en la carpeta PROCESSED
    df_clima.to_csv(OUTPUT_FILE)
    print(f"Dataset de clima PROCESADO guardado exitosamente en: {OUTPUT_FILE}")
else:
    logger.error("No se pudo guardar el archivo porque el DataFrame está vacío.")

Dataset de clima PROCESADO guardado exitosamente en: /Users/pancakes/Documents/Maestria Ciencia de Datos/1/Ingenieria de Caracterisitcas/Proyecto Bachometro/Relacion_Baches_Accidentes_HMO/data/processed/clima_final_processed.csv
